In [168]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
import datetime
from sklearn.linear_model import LinearRegression

In [204]:
tickers = pd.read_excel('spy list.xlsx')
tickers = list(tickers['Symbol'])

AttributeError: 'Series' object has no attribute 'reaplce'

In [92]:
data = yf.download(tickers,period='10y')

[*********************100%***********************]  502 of 502 completed
[*********************100%***********************]  502 of 502 completed


In [169]:
tickers = data['Adj Close'].dropna(how='all').columns

In [194]:
class BasketSelector:
    
    def __init__(self,price,volume,tickers):
        self.price = price
        self.volume = volume
        self.factor_df = pd.DataFrame(index=tickers,columns = ['52wk','pct_abv','39wk','prc_osc','51wk'])
        
    def _momentum_factors(self):
        
        #52 week trend using linear regression
        t1 = self.price.replace(np.nan,1)
        for ticker in tickers:
            #avoid cases of na and less length than expected
            if len(t1) >= 252:
                self.factor_df['52wk']=np.polyfit(range(252),t1[-252:],deg=1)[0]
            elif TypeError:
                self.factor_df.drop(columns = '52wk')
                pass
            else:
                self.factor_df['52wk']=np.polyfit(range(len(t1)),t1,deg=1)[0]
        
        #260 day low
        self.low_price = self.price.rolling(260).min()
        self.factor_df['pct_abv'] = ((self.price-self.low_price)/self.low_price)[-20:].mean()
        
        #39 wk return
        self.factor_df['39wk'] = self.price.pct_change(int(260*0.75))[-20:].mean()

        #4/52 oscillator
        self.four_wk_high = self.price.rolling(20).max()
        self.four_wk_low = self.price.rolling(20).min()

        self.f_wk_high = self.price.rolling(260).max()
        self.f_wk_low = self.price.rolling(260).min()

        self.factor_df['prc_osc'] = ((self.four_wk_high-self.four_wk_low)/(self.f_wk_high-self.f_wk_low))[-20:].mean()
        
        #51 week volume price trend
        t2 = self.volume.replace(np.nan,1)
        for ticker in tickers:
            self.factor_df['51wk']=np.polyfit(range(247),t2[-247:],deg=1)[0]     
        
        return self.factor_df
    
    def basket_outputs(self):
        self.factors = self._momentum_factors()
        #z-score and long basket
        self.z_score = ((self.factors - self.factors.mean())/self.factors.std()).sum(axis=1) 
        return list(self.z_score.nlargest(10).index),list(self.z_score.nsmallest(10).index)

In [195]:
price = data['Adj Close']
#backtest date:
date_list = pd.date_range('2017-9-01','2022-07-01', 
              freq='MS').strftime("%Y-%m-%d").tolist()


In [233]:
monthly = pd.DataFrame(columns = ['Long','Short','Monthly Long Return', 'Monthly Short Return','Average Return','ETF Return'],
                      index = date_list)
long = []
short = []
lret = []
sret = []
aret = []
etf = []

#Monthly return
for date in monthly.index:
    BS = BasketSelector(price=price[price.index<date],volume=data['Volume'],tickers=tickers)
    long1,short1 = BS.basket_outputs()
    x = []
    y = []
    for symbol in short1:
        x.append(symbol[0])
    short.append(x)
    for symbol in long1:
        y.append(symbol[0])
    long.append(y)
    lret.append(float(price[price.index<date][y].pct_change(20)[-1:].mean(axis=1)))
    sret.append(float(price[price.index<date][x].pct_change(20)[-1:].mean(axis=1)))
    aret.append(float(price[price.index<date][x+y].pct_change(20)[-1:].mean(axis=1)))
    etf.append(float(price[price.index<date].pct_change(20)[-1:].mean(axis=1)))


In [234]:
monthly['Long'] = long
monthly['Short']=short
monthly['Monthly Long Return'] = lret
monthly['Monthly Short Return'] = sret
monthly['Average Return'] = aret
monthly['ETF Return'] = etf

In [235]:
monthly

,Long,Short,Monthly Long Return,Monthly Short Return,Average Return,ETF Return
2017-09-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",NaN,NaN,NaN,NaN
2017-10-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.014862,0.014329,0.014595,0.030148
2017-11-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",-0.009984,0.007481,-0.001251,0.016920
2017-12-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.026364,0.020629,0.023496,0.040900
2018-01-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.011905,0.011015,0.011460,0.010032
2018-02-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.017889,0.021175,0.019532,0.041199
2018-03-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",-0.069366,-0.032111,-0.050739,-0.037045
2018-04-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",-0.021492,-0.010167,-0.015830,0.005771
2018-05-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.066803,0.029905,0.048354,0.024887
2018-06-01,"[DIS, AMCR, FFIV, NLSN, CI, VZ, MTD, BKR, SYF,...","[PAYX, J, NI, ABC, LIN, NWL, CHD, ARE, ZBRA, HLT]",0.005141,0.020295,0.012718,0.028331
